In [ ]:
from repenseai.genai.agent import Agent
from repenseai.genai.tasks.api import Task

from pydantic import BaseModel

# Define Pydantic model for evaluation response
class EvaluationResult(BaseModel):
    satisfied: bool
    score: int
    feedback: str
    improvement_suggestions: str

# Create the agents
generator_agent = Agent(
    model="writer/palmyra-med-70b",
    model_type="chat",
)

evaluator_agent = Agent(
    model="gpt-4o",
    model_type="chat",
    json_schema=EvaluationResult
)

# Define the prompts
GENERATOR_PROMPT = """
You are a professional content creator. Your task is to write a compelling blog post introduction 
on the topic of "Artificial Intelligence in Healthcare".

If this is the first attempt, write a creative and engaging introduction.
If this is a revision, use the feedback provided to improve your introduction.

Previous attempt: {previous_attempt}
Feedback: {feedback}
Improvement suggestions: {improvement_suggestions}

Write 2-3 paragraphs that will captivate the reader's interest and establish the importance 
of AI in modern healthcare.
"""

EVALUATOR_PROMPT = """
You are a professional content editor evaluating a blog post introduction on 
"Artificial Intelligence in Healthcare".

Introduction:
---
{content}
---

Provide a thorough evaluation based on these criteria:
1. Clarity: Is the introduction clear and easy to understand?
2. Engagement: Does it capture the reader's interest?
3. Relevance: Does it effectively introduce the topic of AI in healthcare?
4. Professionalism: Is the tone and language appropriate for the audience?
5. Accuracy: Are the statements factually correct?

Return your evaluation in the following JSON format:
{
  "satisfied": true/false (whether this introduction meets professional standards),
  "score": (a score from 1-10),
  "feedback": "General feedback on the strengths and weaknesses",
  "improvement_suggestions": "Specific suggestions for improvement"
}

Only return "satisfied": true if the content is truly excellent (score 8 or higher) and requires 
no significant improvements.
"""

# Topic and initial conditions
previous_attempt = ""
feedback = ""
improvement_suggestions = ""

max_iterations = 3
current_iteration = 0
is_satisfied = False

print(f"Maximum iterations: {max_iterations}\n")

while current_iteration < max_iterations and not is_satisfied:
    current_iteration += 1
    print(f"\n--- ITERATION {current_iteration} ---\n")

    context = {
        "previous_attempt": previous_attempt if previous_attempt else "This is the first attempt.",
        "feedback": feedback if feedback else "No feedback yet.",
        "improvement_suggestions": improvement_suggestions if improvement_suggestions else "No suggestions yet.",
    }
    
    # 1. Generator Task - Create or improve content
    generator_task = Task(
        user=GENERATOR_PROMPT,
        agent=generator_agent,
        simple_response=True,
    )
    
    # Run the generator task
    generator_result = generator_task.run(context)
    context['content'] = generator_result
    
    print("GENERATED CONTENT:")
    print("-" * 50)
    print(generator_result)
    print("-" * 50)
    
    # 2. Evaluator Task - Evaluate the content
    evaluator_task = Task(
        user=EVALUATOR_PROMPT,
        agent=evaluator_agent,
        simple_response=True,
    )
    
    # Run the evaluator task
    evaluator_result = evaluator_task.run(context)
    
    print("\nEVALUATION RESULTS:")
    print(f"Satisfied: {evaluator_result['satisfied']}")
    print(f"Score: {evaluator_result['score']}/10")
    print(f"Feedback: {evaluator_result['feedback']}")
    print(f"Improvement Suggestions: {evaluator_result['improvement_suggestions']}")
    
    # Check if evaluator is satisfied or if we need another iteration
    
    if evaluator_result['satisfied']:
        print(f"\n✅ EVALUATOR SATISFIED ON ITERATION {current_iteration}")
        print("Final content:")
        print("-" * 50)
        print(generator_result)
        print("-" * 50)
        break
    elif current_iteration >= max_iterations:
        print(f"\n⚠️ REACHED MAXIMUM ITERATIONS ({max_iterations})")
        print(f"Best content (score: {evaluator_result['score']}/10):")
        print("-" * 50)
        print(generator_result)
        print("-" * 50)
    else:
        print(f"\n🔄 PROCEEDING TO ITERATION {current_iteration + 1}...")
        # Update variables for next iteration
        previous_attempt = generator_result
        feedback = evaluator_result['feedback']
        improvement_suggestions = evaluator_result['improvement_suggestions']